In [1]:
from easyAI.AI import NegamaxNoPruning
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax, NegamaxNoPruning, NonRecursiveNegamax, SSS, Expectiminimax
import random
from tqdm.auto import tqdm
import pandas as pd

In [2]:
class Nim(TwoPlayerGame):
    def __init__(self, players, piles = [5, 5, 5, 5], deterministic = False):
        self.piles = piles
        self.players = players
        self.nplayer = 1
        self.current_player = 1
        self.deterministic = deterministic
    
    def possible_moves(self):
        moves = ["%d,%d" % (i + 1, j) for i in range(len(self.piles)) for j in range(1, self.piles[i] + 1)]
        return moves
    
    def make_final_move(self, move):
        pile, count = move.split(",")
        extra = 1 if random.randint(1, 10) == 1 and not self.deterministic else 0
        self.piles[int(pile) - 1] -= int(count) - extra
        return extra
    
    def make_move(self, move):
        pile, count = move.split(",")
        self.piles[int(pile) - 1] -= int(count)

    def unmake_move(self, move): # optional method (speeds up the AI)
        pile, count = move.split(",")
        self.piles[int(pile) - 1] += int(count)

    def show(self): print(" ".join(map(str, self.piles)))

    def win(self): return max(self.piles) == 0

    def is_over(self): return self.win()

    def scoring(self): return 100 if self.win() else 0

    def ttentry(self): return tuple(self.piles) #optional, speeds up AI

    def ttrestore(self, entry): self.piles = list(entry)

# Negamax

In [6]:
# Win tracking dict
results = {}

depths = [3, 7]
deterministic = [True, False]
game_amount = 10

for d1, d2 in tqdm([(d1, d2) for d1 in depths for d2 in depths], desc="Processing depths"):
    for det in tqdm(deterministic, desc="Processing deterministic"):
        # print(f"depth1: {d1}, depth2: {d2} deterministic: {det}")
        
        # Play 10 games and track the results
        p1_wins = 0
        p2_wins = 0
        p1_avg_time = 0
        p2_avg_time = 0

        for x in tqdm(range(game_amount), desc="Playing games"):
            ai = Negamax(d1)
            ai2 = Negamax(d2)
            game = Nim(players=[AI_Player(ai), AI_Player(ai2)],
                    deterministic=det,
                    piles=[5,5,5,5])
            p1_time, p2_time = game.play(verbose=False)
            p1_avg_time += p1_time
            p2_avg_time += p2_time
            # print("\nplayer %d wins\n" % game.current_player)
            if game.current_player == 1:
                p1_wins += 1
            else:
                p2_wins += 1 

        p1_avg_time /= game_amount
        p2_avg_time /= game_amount
        results[(d1, d2, det)] = {"p1_wins": p1_wins,
                                    "p2_wins": p2_wins,
                                    "p1_average_game_time": p1_avg_time,
                                    "p2_average_game_time": p2_avg_time} # Update the results

print(results)

Processing depths:   0%|          | 0/4 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

...

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

{(3, 3, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.00032284061999973804, 'p2_average_game_time': 0.00032643342221629404}, (3, 3, False): {'p1_wins': 1, 'p2_wins': 9, 'p1_average_game_time': 0.00037722063863429177, 'p2_average_game_time': 0.0003784540660614073}, (3, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.00030049969999652144, 'p2_average_game_time': 0.02232468786666636}, (3, 7, False): {'p1_wins': 3, 'p2_wins': 7, 'p1_average_game_time': 0.00031671366863765695, 'p2_average_game_time': 0.02276145709454843}, (7, 3, True): {'p1_wins': 10, 'p2_wins': 0, 'p1_average_game_time': 0.028406229699998498, 'p2_average_game_time': 0.00032280886666676966}, (7, 3, False): {'p1_wins': 7, 'p2_wins': 3, 'p1_average_game_time': 0.025529068405403005, 'p2_average_game_time': 0.00029152145737249165}, (7, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.023933521789999194, 'p2_average_game_time': 0.021653484222219452}, (7, 7, False): {'p1_wins': 

## TODO:
wziac rzeczy z results i se zrobic plota jakiegos fajnego

# Negamax bez alpha-beta pruning
(zakładając, że copilot dobrze zaimplementował (ale chyba jest git xd))

In [5]:
# Win tracking dict
results = {}

depths = [3, 7]
deterministic = [True, False]
game_amount = 10

for d1, d2 in tqdm([(d1, d2) for d1 in depths for d2 in depths], desc="Processing depths"):
    for det in tqdm(deterministic, desc="Processing deterministic"):
        # print(f"depth1: {d1}, depth2: {d2} deterministic: {det}")
        
        # Play 10 games and track the results
        p1_wins = 0
        p2_wins = 0
        p1_avg_time = 0
        p2_avg_time = 0

        for x in tqdm(range(game_amount), desc="Playing games"):
            ai = NegamaxNoPruning(d1)
            ai2 = NegamaxNoPruning(d2)
            game = Nim(players=[AI_Player(ai), AI_Player(ai2)],
                    deterministic=det,
                    piles=[5,5,5,5])
            p1_time, p2_time = game.play(verbose=False)
            p1_avg_time += p1_time
            p2_avg_time += p2_time
            # print("\nplayer %d wins\n" % game.current_player)
            if game.current_player == 1:
                p1_wins += 1
            else:
                p2_wins += 1 

        p1_avg_time /= game_amount
        p2_avg_time /= game_amount
        results[(d1, d2, det)] = {"p1_wins": p1_wins,
                                    "p2_wins": p2_wins,
                                    "p1_average_game_time": p1_avg_time,
                                    "p2_average_game_time": p2_avg_time} # Update the results

print(results)

Processing depths:   0%|          | 0/4 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

...

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Non Recursive Negamax

In [6]:
# Win tracking dict
results = {}

depths = [3, 7]
deterministic = [True, False]
game_amount = 10

for d1, d2 in tqdm([(d1, d2) for d1 in depths for d2 in depths], desc="Processing depths"):
    for det in tqdm(deterministic, desc="Processing deterministic"):
        # print(f"depth1: {d1}, depth2: {d2} deterministic: {det}")
        
        # Play 10 games and track the results
        p1_wins = 0
        p2_wins = 0
        p1_avg_time = 0
        p2_avg_time = 0

        for x in tqdm(range(game_amount), desc="Playing games"):
            ai = NonRecursiveNegamax(d1)
            ai2 = NonRecursiveNegamax(d2)
            game = Nim(players=[AI_Player(ai), AI_Player(ai2)],
                    deterministic=det,
                    piles=[5,5,5,5])
            p1_time, p2_time = game.play(verbose=False)
            p1_avg_time += p1_time
            p2_avg_time += p2_time
            # print("\nplayer %d wins\n" % game.current_player)
            if game.current_player == 1:
                p1_wins += 1
            else:
                p2_wins += 1 

        p1_avg_time /= game_amount
        p2_avg_time /= game_amount
        results[(d1, d2, det)] = {"p1_wins": p1_wins,
                                    "p2_wins": p2_wins,
                                    "p1_average_game_time": p1_avg_time,
                                    "p2_average_game_time": p2_avg_time} # Update the results

print(results)

Processing depths:   0%|          | 0/4 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

...

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

{(3, 3, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.0005866637700012233, 'p2_average_game_time': 0.0005696205333341216}, (3, 3, False): {'p1_wins': 5, 'p2_wins': 5, 'p1_average_game_time': 0.000642666206672712, 'p2_average_game_time': 0.0005507926494922713}, (3, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.0005234869399998842, 'p2_average_game_time': 0.033736884922223}, (3, 7, False): {'p1_wins': 2, 'p2_wins': 8, 'p1_average_game_time': 0.0005205774236326408, 'p2_average_game_time': 0.03318309763343451}, (7, 3, True): {'p1_wins': 10, 'p2_wins': 0, 'p1_average_game_time': 0.030836591211105872, 'p2_average_game_time': 0.00034715641110728433}, (7, 3, False): {'p1_wins': 8, 'p2_wins': 2, 'p1_average_game_time': 0.030318803274541607, 'p2_average_game_time': 0.0003519581877825809}, (7, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.028017885389996312, 'p2_average_game_time': 0.023945174255559067}, (7, 7, False): {'p1_wins': 4, 'p2_wi

# SSS

In [8]:
# Win tracking dict
results = {}

depths = [3, 7]
deterministic = [True, False]
game_amount = 10

for d1, d2 in tqdm([(d1, d2) for d1 in depths for d2 in depths], desc="Processing depths"):
    for det in tqdm(deterministic, desc="Processing deterministic"):
        # print(f"depth1: {d1}, depth2: {d2} deterministic: {det}")
        
        # Play 10 games and track the results
        p1_wins = 0
        p2_wins = 0
        p1_avg_time = 0
        p2_avg_time = 0

        for x in tqdm(range(game_amount), desc="Playing games"):
            ai = SSS(d1)
            ai2 = SSS(d2)
            game = Nim(players=[AI_Player(ai), AI_Player(ai2)],
                    deterministic=det,
                    piles=[5,5,5,5])
            p1_time, p2_time = game.play(verbose=False)
            p1_avg_time += p1_time
            p2_avg_time += p2_time
            # print("\nplayer %d wins\n" % game.current_player)
            if game.current_player == 1:
                p1_wins += 1
            else:
                p2_wins += 1 

        p1_avg_time /= game_amount
        p2_avg_time /= game_amount
        results[(d1, d2, det)] = {"p1_wins": p1_wins,
                                    "p2_wins": p2_wins,
                                    "p1_average_game_time": p1_avg_time,
                                    "p2_average_game_time": p2_avg_time} # Update the results

print(results)

Processing depths:   0%|          | 0/4 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

...

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

{(3, 3, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.00048039088000336957, 'p2_average_game_time': 0.00046510045554997413}, (3, 3, False): {'p1_wins': 4, 'p2_wins': 6, 'p1_average_game_time': 0.0005971208192443445, 'p2_average_game_time': 0.0005542256237402402}, (3, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.0004740234799885456, 'p2_average_game_time': 0.04528161925555625}, (3, 7, False): {'p1_wins': 2, 'p2_wins': 8, 'p1_average_game_time': 0.000454358709845475, 'p2_average_game_time': 0.04621399053909251}, (7, 3, True): {'p1_wins': 10, 'p2_wins': 0, 'p1_average_game_time': 0.038623570177771356, 'p2_average_game_time': 0.0003055983555491871}, (7, 3, False): {'p1_wins': 8, 'p2_wins': 2, 'p1_average_game_time': 0.03717148480313436, 'p2_average_game_time': 0.00031215994243158926}, (7, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.04382637860000358, 'p2_average_game_time': 0.039978993828573756}, (7, 7, False): {'p1_wins': 2, 'p2_

# Expectiminimax

In [3]:
# Win tracking dict
results = {}

depths = [3, 7]
deterministic = [True, False]
game_amount = 10

for d1, d2 in tqdm([(d1, d2) for d1 in depths for d2 in depths], desc="Processing depths"):
    for det in tqdm(deterministic, desc="Processing deterministic"):
        # print(f"depth1: {d1}, depth2: {d2} deterministic: {det}")
        
        # Play 10 games and track the results
        p1_wins = 0
        p2_wins = 0
        p1_avg_time = 0
        p2_avg_time = 0

        for x in tqdm(range(game_amount), desc="Playing games"):
            ai = Expectiminimax(d1)
            ai2 = Expectiminimax(d2)
            game = Nim(players=[AI_Player(ai), AI_Player(ai2)],
                    deterministic=det,
                    piles=[5,5,5,5])
            p1_time, p2_time = game.play(verbose=False)
            p1_avg_time += p1_time
            p2_avg_time += p2_time
            # print("\nplayer %d wins\n" % game.current_player)
            if game.current_player == 1:
                p1_wins += 1
            else:
                p2_wins += 1 

        p1_avg_time /= game_amount
        p2_avg_time /= game_amount
        results[(d1, d2, det)] = {"p1_wins": p1_wins,
                                    "p2_wins": p2_wins,
                                    "p1_average_game_time": p1_avg_time,
                                    "p2_average_game_time": p2_avg_time} # Update the results

print(results)

Processing depths:   0%|          | 0/4 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

...

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

{(3, 3, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.00044623370016779514, 'p2_average_game_time': 0.0004422345778948511}, (3, 3, False): {'p1_wins': 5, 'p2_wins': 5, 'p1_average_game_time': 0.002675226461293884, 'p2_average_game_time': 0.0025235621283643166}, (3, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.000316273500029638, 'p2_average_game_time': 0.022827188000054677}, (3, 7, False): {'p1_wins': 3, 'p2_wins': 7, 'p1_average_game_time': 0.0022077372960043386, 'p2_average_game_time': 3.809995472198768}, (7, 3, True): {'p1_wins': 10, 'p2_wins': 0, 'p1_average_game_time': 0.029628635400048446, 'p2_average_game_time': 0.00032066074434422055}, (7, 3, False): {'p1_wins': 5, 'p2_wins': 5, 'p1_average_game_time': 4.283704106790841, 'p2_average_game_time': 0.002106719716818653}, (7, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.026408791950052544, 'p2_average_game_time': 0.02294283265548276}, (7, 7, False): {'p1_wins': 5, 'p2_wins':

In [3]:
# read txt, it contains a dict, assign it to a variable
with open("results/Negamax.txt", "r") as file:
    negamax_results = eval(file.read())
with open("results/NegamaxNoPruning.txt", "r") as file:
    negamax_no_pruning_results = eval(file.read())
with open("results/NonRecursiveNegamax.txt", "r") as file:
    non_recursive_negamax_results = eval(file.read())
with open("results/SSS.txt", "r") as file:
    sss_results = eval(file.read())
with open("results/Expectiminimax.txt", "r") as file:
    expectiminimax_results = eval(file.read())

In [4]:
negamax_results.keys()

dict_keys([(3, 3, True), (3, 3, False), (3, 7, True), (3, 7, False), (7, 3, True), (7, 3, False), (7, 7, True), (7, 7, False)])

In [5]:
def prepare_dataframe(results, det, algorithm_name):
    # Only take keys with deterministic = det, also remove the deterministic key
    results = {k: v for k, v in results.items() if k[2] == det}
    results = {k[:2]: v for k, v in results.items()}

    # Convert to dataframes
    df = pd.DataFrame(results)
    # Swap rows with columns
    df = df.transpose()
    # Change column names
    df.columns = ["P1 wins", "P2 wins", "P1 average turn time", "P2 average turn time"]
    # Add column for the depth
    df["Depth"] = df.index
    # Convert float to int
    df["P1 wins"] = df["P1 wins"].astype(int)
    df["P2 wins"] = df["P2 wins"].astype(int)
    # Add column for the algorithm name
    df["Algorithm"] = algorithm_name
    return df

In [8]:
# Append all deterministic dataframes together
df_deterministic = pd.concat([prepare_dataframe(negamax_results, True, "Negamax"),
                prepare_dataframe(negamax_no_pruning_results, True, "NegamaxNoPruning"),
                prepare_dataframe(non_recursive_negamax_results, True, "NonRecursiveNegamax"),
                prepare_dataframe(sss_results, True, "SSS"),
                prepare_dataframe(expectiminimax_results, True, "Expectiminimax")])

df_deterministic.reset_index(drop=True, inplace=True)
df_deterministic

,P1 wins,P2 wins,P1 average turn time,P2 average turn time,Depth,Algorithm
0,0,10,0.000323,0.000326,"(3, 3)",Negamax
1,0,10,0.000300,0.022325,"(3, 7)",Negamax
2,10,0,0.028406,0.000323,"(7, 3)",Negamax
3,0,10,0.023934,0.021653,"(7, 7)",Negamax
4,0,10,0.002352,0.002235,"(3, 3)",NegamaxNoPruning
5,0,10,0.002285,6.192901,"(3, 7)",NegamaxNoPruning
6,10,0,9.457784,0.002167,"(7, 3)",NegamaxNoPruning
7,0,10,8.437451,6.304129,"(7, 7)",NegamaxNoPruning
8,0,10,0.000587,0.000570,"(3, 3)",NonRecursiveNegamax
9,0,10,0.000523,0.033737,"(3, 7)",NonRecursiveNegamax


In [10]:
# Append all non-deterministic dataframes together
df_non_deterministic = pd.concat([prepare_dataframe(negamax_results, False, "Negamax"),
                prepare_dataframe(negamax_no_pruning_results, False, "NegamaxNoPruning"),
                prepare_dataframe(non_recursive_negamax_results, False, "NonRecursiveNegamax"),
                prepare_dataframe(sss_results, False, "SSS"),
                prepare_dataframe(expectiminimax_results, False, "Expectiminimax")])

df_non_deterministic.reset_index(drop=True, inplace=True)
df_non_deterministic

,P1 wins,P2 wins,P1 average turn time,P2 average turn time,Depth,Algorithm
0,1,9,0.000377,0.000378,"(3, 3)",Negamax
1,3,7,0.000317,0.022761,"(3, 7)",Negamax
2,7,3,0.025529,0.000292,"(7, 3)",Negamax
3,5,5,0.024704,0.021459,"(7, 7)",Negamax
4,6,4,0.002165,0.001970,"(3, 3)",NegamaxNoPruning
5,4,6,0.002478,6.702793,"(3, 7)",NegamaxNoPruning
6,8,2,9.432900,0.002168,"(7, 3)",NegamaxNoPruning
7,5,5,9.085430,7.044955,"(7, 7)",NegamaxNoPruning
8,5,5,0.000643,0.000551,"(3, 3)",NonRecursiveNegamax
9,2,8,0.000521,0.033183,"(3, 7)",NonRecursiveNegamax


In [18]:
# Convert to csv
df_deterministic.to_csv("results/deterministic.csv", index=False, sep=";")
df_non_deterministic.to_csv("results/non_deterministic.csv", index=False, sep=";")